In [102]:
import numpy as np
import pandas as pd

In [103]:
poems_raw = pd.read_csv('./data/shakespeare.txt', sep='\t', header=None, names=['text'])

In [292]:
# remove rows containing numbers (the number of each poem), and reset indices to account for deleted rows
poems_proc = poems_raw[~poems_raw['text'].str.contains(r'\d')]
poems_proc.reset_index(drop=True, inplace=True)

# replace characters which are not apostrophes, spaces, dashes, or letters
poems_proc['text'] = poems_proc['text'].str.replace('[^\'\s\-a-zA-Z]', '')

# make all characters lowercase
poems_proc['text'] = poems_proc['text'].str.lower()


/Users/slushtak/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/slushtak/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [107]:
# add poem and row (within each poem) numbers
poems_proc['poem'] = (poems_proc.index / 14).astype(int)
poems_proc['line'] = (poems_proc.index - poems_proc['poem']*14).astype(int)


/Users/slushtak/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/slushtak/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [108]:
poems_proc.head(30)

,text,poem,line
0,from fairest creatures we desire increase,0,0
1,that thereby beauty's rose might never die,0,1
2,but as the riper should by time decease,0,2
3,his tender heir might bear his memory,0,3
4,but thou contracted to thine own bright eyes,0,4
5,feed'st thy light's flame with self-substantia...,0,5
6,making a famine where abundance lies,0,6
7,thy self thy foe to thy sweet self too cruel,0,7
8,thou that art now the world's fresh ornament,0,8
9,and only herald to the gaudy spring,0,9


In [270]:
# efficiently create words dataframe by first creating list-of-lists,
# as you cannot append to dataframe in-place
words_list = []

for _,row in poems_proc.iterrows():
    line_index = 0
    for word in row['text'].split():
        # if last word in the line
        if word == row['text'].split()[-1]:
            words_list.append([word, row['poem'], row['line'], line_index, True])
        else:
            words_list.append([word, row['poem'], row['line'], line_index, False])
        line_index += 1

words_raw = pd.DataFrame.from_records(words_list, columns=['word', 'poem', 'line', 'line_idx', 'line_end'])

In [271]:
words_raw

,word,poem,line,line_idx,line_end
0,from,0,0,0,False
1,fairest,0,0,1,False
2,creatures,0,0,2,False
3,we,0,0,3,False
4,desire,0,0,4,False
...,...,...,...,...,...
17577,water,153,12,3,False
17578,water,153,12,4,False
17579,cools,153,12,5,False
17580,not,153,12,6,False


In [272]:
syllables_list = []

with open('./data/Syllable_dictionary.txt', newline='') as file:
    for l in file:
        line = l.split()
        
        # if it has a different syllable count when at end of line
        # (1st character of 2nd elt of line is an E)
        if line[1][0] == 'E':
            syllables_list.append([ line[0], int(line[1][1:]), int(line[2]) ])

        # though most entries look like 'word E1 2', some (annoyingly) are 'word 2 E1'
        elif line[-1][0] == 'E':
            syllables_list.append([ line[0], int(line[-1][1:]), int(line[1]) ])
        
        else:
            # TODO: case where word has 2 options for number of syllables
            syllables_list.append([ line[0], int(line[-1]), int(line[-1]) ])

syllables = pd.DataFrame.from_records(syllables_list, columns=['word', 'end_syl', 'syl'])

In [273]:
words = words_raw.join(syllables.set_index('word'), on='word')

# there are some words without syllable counts; this appears to be due to
# single quotes being used as quotes, or a possessive at the end of a word,
# rather than apostrophes within words

words2 = words[words['syl'].isnull()][['word', 'poem', 'line', 'line_idx']]
words2['word'] = words2['word'].str.replace('\'', '')
words3 = words2.join(syllables.set_index('word'), on='word')
words.update(words3)

In [274]:
words.head(30)

,word,poem,line,line_idx,line_end,end_syl,syl
0,from,0.0,0.0,0.0,False,1.0,1.0
1,fairest,0.0,0.0,1.0,False,2.0,2.0
2,creatures,0.0,0.0,2.0,False,2.0,2.0
3,we,0.0,0.0,3.0,False,1.0,1.0
4,desire,0.0,0.0,4.0,False,2.0,2.0
5,increase,0.0,0.0,5.0,True,2.0,2.0
6,that,0.0,1.0,0.0,False,1.0,1.0
7,thereby,0.0,1.0,1.0,False,2.0,2.0
8,beauty's,0.0,1.0,2.0,False,2.0,2.0
9,rose,0.0,1.0,3.0,False,1.0,1.0
